In [1]:
import sys
sys.path.append('../')

import os
import json
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from scipy.stats import gmean
from phoenix.synthesis import grouping, ordering
from phoenix.models import HamiltonianModel, Clifford2Q, BSF
from phoenix import Circuit
from phoenix import models
from phoenix.utils import passes

from rustiq import pauli_network_synthesis
from rustiq import Metric

In [24]:
paulis_a, coeffs_a = ['XXXZIYZI', 'YXXZIYYI', 'ZXXZIYZI'], np.array([-0.0125, -0.0125, -0.0125])
# paulis_b, coeffs_b = ['IYZZXXZI', 'IYXZYXYI', 'IYYZXXYI', 'IYXZZXZI'], np.array([-0.0125, -0.0125,  0.0125, -0.0125])
cliffs = pauli_network_synthesis(paulis_a, Metric.COUNT)

In [35]:
tab = BSF(paulis_a)

for c in cliffs:
    # print('{} on {}'.format(c, tab.paulis))

    if c[0] == 'CNOT':
        tab = tab.apply_cx(c[1][0], c[1][1])
    elif c[0] == 'H':
        tab = tab.apply_h(c[1][0])
    elif c[0] == 'S':
        tab = tab.apply_s(c[1][0])
    
    popped = tab.pop_local_paulis()
    if popped.num_paulis > 0:
        print('popped: {}'.format(popped.paulis))
    print('now: {}'.format(tab.paulis))

now: ['XIXZIYZI', 'YIXZIYYI', 'ZXXZIYZI']
now: ['XIIZIYZI', 'YIIZIYYI', 'ZXXZIYZI']
now: ['ZIIZIYZI', 'YIIZIYYI', 'XXXZIYZI']
now: ['ZIIIIYZI', 'YIIIIYYI', 'XXXZIYZI']
now: ['XIIIIYZI', 'YIIIIYYI', 'ZXXZIYZI']
now: ['XIIIIXZI', 'YIIIIXYI', 'ZXXZIXZI']
now: ['XIIIIIZI', 'YIIIIIYI', 'ZXXZIXZI']
now: ['ZIIIIIZI', 'YIIIIIYI', 'XXXZIXZI']
popped: ['ZIIIIIII']
now: ['ZIIIIIXI', 'XXXZIXZI']
now: ['XIIIIIXI', 'ZXXZIXZI']
popped: ['IIIIIIXI']
now: ['ZXXZIXII']
now: ['XXXZIXII']
now: ['IXXZIXII']
now: ['IIXZIXII']
now: ['IIZZIXII']
now: ['IIZIIXII']
now: ['IIXIIXII']
popped: ['IIIIIXII']
now: []


[]

In [16]:
tab.mat[:, [idx, idx + tab.num_qubits]]

array([[1, 0],
       [1, 1],
       [0, 1]])

In [2]:
benchmark_dpath = '../benchmarks/uccsd_json'
fnames = [os.path.join(benchmark_dpath, fname) for fname in natsorted(os.listdir(benchmark_dpath))]
num_programs = len(fnames)
programs = [os.path.basename(fname).split('.')[0] for fname in fnames]

result = pd.read_csv('./results/result_uccsd_phoenix.csv')[['program', 'num_qubits', 'num_2q_gates', 'depth_2q', 'num_2q_gates(all2all)', 'depth_2q(all2all)', 'num_2q_gates(all2all_opt)', 'depth_2q(all2all_opt)']]
result = result.rename({'num_2q_gates(all2all)': 'num_2q_gates(phoenix)', 'depth_2q(all2all)': 'depth_2q(phoenix)', 'num_2q_gates(all2all_opt)': 'num_2q_gates(phoenix_opt)', 'depth_2q(all2all_opt)': 'depth_2q(phoenix_opt)'}, axis=1)

In [3]:
result_ablation = pd.DataFrame(columns=['program', 'num_2q_gates(phoenix_bsf)', 'depth_2q(phoenix_bsf)', 'num_2q_gates(phoenix_bsf_order)', 'depth_2q(phoenix_bsf_order)'])
for fname in tqdm(fnames):
    program_name = os.path.basename(fname).split('.')[0]
    with open(fname, 'r') as f:
        data = json.load(f)
    ham = HamiltonianModel(data['paulis'], data['coeffs'])
    if ham.num_qubits > 10:
        continue
    circ_bsf = ham.phoenix_circuit(order=False)
    circ_bsf_order = ham.phoenix_circuit(efficient=True)

    result_ablation = pd.concat([result_ablation, pd.DataFrame({
        'program': program_name,
        'num_2q_gates(phoenix_bsf)': circ_bsf.num_nonlocal_gates,
        'depth_2q(phoenix_bsf)': circ_bsf.depth_nonlocal,
        'num_2q_gates(phoenix_bsf_order)': circ_bsf_order.num_nonlocal_gates,
        'depth_2q(phoenix_bsf_order)': circ_bsf_order.depth_nonlocal
    }, index=[0])], ignore_index=True)
    break


 62%|██████▎   | 10/16 [00:09<00:05,  1.04it/s]


In [4]:
result_ablation

,program,num_2q_gates(phoenix_bsf),depth_2q(phoenix_bsf),num_2q_gates(phoenix_bsf_order),depth_2q(phoenix_bsf_order)
0,LiH_frz_BK_sto3g,460,428,452,403


In [5]:
result

,program,num_qubits,num_2q_gates,depth_2q,num_2q_gates(phoenix),depth_2q(phoenix),num_2q_gates(phoenix_opt),depth_2q(phoenix_opt)
0,CH2_cmplt_BK,14,19574,19399,4112,3483,3870,3253
1,CH2_cmplt_JW,14,21072,19749,4150,3636,3680,3040
2,CH2_frz_BK,12,10228,10174,2214,1909,2102,1804
3,CH2_frz_JW,12,10344,9706,2202,1973,1987,1683
4,H2O_cmplt_BK,14,13108,12976,2810,2406,2612,2216
5,H2O_cmplt_JW,14,14360,13576,2808,2514,2471,2085
6,H2O_frz_BK,12,8004,7934,1418,1195,1356,1137
7,H2O_frz_JW,12,8064,7613,1740,1564,1560,1326
8,LiH_cmplt_BK,12,8680,8637,1484,1285,1414,1218
9,LiH_cmplt_JW,12,8064,7616,1702,1535,1532,1305
